# :star: Unified Equity Research Chatbot :dollar:
## :snowflake: Created Using [Snowflake Cortex Agent](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-agents) :snowflake: :rocket: :rocket: 
- Cortex Agents orchestrate across both structured and unstructured data sources to deliver insights. 
- They plan tasks, use tools to execute these tasks, and generate responses. 
- Agents use Cortex Analyst (structured) and Cortex Search (unstructured) as tools, along with LLMs, to analyze data. 
- Cortex Search extracts insights from unstructured sources, while Cortex Analyst generates SQL to process structured data. 
- A comprehensive support for tool identification and tool execution enables delivery of sophisticated applications grounded in enterprise data.

(Source: [Snowflake Documentation](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-agents))

# The steps we will execute in this Notebook and in the Streamlit App for you to test Cortex Agent.  
## Phase 1: Ingesting & Transforming OHLC JSON data.  
### 1. Ingested OHLC JSON data into Snowflake.
### 2. Transform the JSON Data using FLATTEN and CAST the data to the correct data type.
### 3. Created a VIEW on the flattened data. 
### 4. Created a Cortex Analyst Semantic Model on the View. 
### 5. Tested the semantic model with a few questions.
### 6. Received tabular data from Cortex Analyst.
## Phase 2: Ingest, Parse, Chunk, and Create Cortex Search Service on PDF documents.    
### 1. Create a stage for the company-related PDF files.
### 2. Store the company annual report PDF into a stage. 
### 3. Parse the PDF files in the stage and extract the text. 
### 4. Store the text in a VARCHAR column in a table. 
### 5. Split the text into chunks. 
### 6. Create Snowflake Cortex Search service on the chunks.
### 7. Test the Cortex Search Service to see it returns the correct results.  
## Phase 3: Create a Streamlit App to Test Cortex Agent.
### 1. Set-up Snowflake Cortex Agent in Streamlit in Snowflake
### 2. Ask questions & get answers. 
### 3. Snowflake Cortex Agent will interpret your question. 
### 4. Route it intelligently to either Cortex Analyst or Cortex Search. 
### 5. If a question is routed to Cortex Analyst, tabular data is received as an answer. 
### 6. If a question is routed to Cortex Search,  textual answer is received as an answer.  

## Create a Snowflake-Managed Stage to Store the OHLC JSON Files
```
CREATE STAGE OHLC_STOCK_PRICES_INTERNAL_STG 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' ) 
	COMMENT = 'Stage to Store the OHLC JSON Data Files for Processing.';
```

In [ ]:
CREATE STAGE OHLC_STOCK_PRICES_INTERNAL_STG 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' ) 
	COMMENT = 'Stage to Store the OHLC JSON Data Files for Processing.';

### Download and Unzip [OHLC_Data.zip](https://github.com/rrprasan/Finance/tree/main/Snowflake/Notebooks/Company_Financials/unified_equity_research) from Github
### You will find six JSON files
### Load the files into OHLC_STOCK_PRICES_INTERNAL_STG
- OHLCdata_AMZN.json
- OHLCdata_CPB.json
- OHLCdata_SJM.json
- OHLCdata_VDE.json
- OHLCdata_VHT.json
- OHLCdata_VNQ.json
#### OHLC Data Provided By [Polygon.io](https://polygon.io/)

## Check that the OHLC JSON Files Exist in the Stage
### List the Files in the Stage

In [ ]:
LIST @OHLC_STOCK_PRICES_INTERNAL_STG

## Create a Table to Load the Raw JSON Data
```
CREATE OR REPLACE TRANSIENT TABLE COMPANY_STOCK_PRICES_DAILY_OHLC_RAW_TBL
(
TICKER VARIANT,
RESULTS VARIANT
);
```

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE COMPANY_STOCK_PRICES_DAILY_OHLC_RAW_TBL
(
TICKER VARIANT,
RESULTS VARIANT
);

## Use the COPY Command to Copy the JSON Data into the Table 
```
COPY INTO COMPANY_STOCK_PRICES_DAILY_OHLC_RAW_TBL
  FROM @OHLC_STOCK_PRICES_INTERNAL_STG
  FILE_FORMAT = (TYPE = 'JSON')
  MATCH_BY_COLUMN_NAME='CASE_INSENSITIVE';
```

In [ ]:
COPY INTO COMPANY_STOCK_PRICES_DAILY_OHLC_RAW_TBL
  FROM @OHLC_STOCK_PRICES_INTERNAL_STG
  FILE_FORMAT = (TYPE = 'JSON')
  MATCH_BY_COLUMN_NAME='CASE_INSENSITIVE';

In [ ]:
SELECT * FROM COMPANY_STOCK_PRICES_DAILY_OHLC_RAW_TBL;

## Use [FLATTEN](https://docs.snowflake.com/en/sql-reference/functions/flatten) SQL Function to explode JSON Compound Values into Multiple Rows.  
### First, test the SQL to ensure it returns data correctly.

In [ ]:
-- FLATTEN & CAST the OHLC Data into NUMBER Data Type
SELECT
    TICKER::VARCHAR                                         TICKER_SYMBOL,
    OHLC_DATA.VALUE:"from_date"::DATE                       OHLC_DATE,
    TO_NUMBER(OHLC_DATA.VALUE:"open",14, 4)                 OPEN_PRICE,
    TO_NUMBER(OHLC_DATA.VALUE:"high",14, 4)                 HIGH_PRICE,
    TO_NUMBER(OHLC_DATA.VALUE:"low",14, 4)                  LOW_PRICE,
    TO_NUMBER(OHLC_DATA.VALUE:"close",14, 4)                CLOSE_PRICE,
FROM 
    COMPANY_STOCK_PRICES_DAILY_OHLC_RAW_TBL CSTR,
    LATERAL FLATTEN (input => CSTR.RESULTS) OHLC_DATA
ORDER BY TICKER_SYMBOL, OHLC_DATE;

## Same query as above with data type cast using :: Notation.

In [ ]:
SELECT
    TICKER::VARCHAR                                         TICKER_SYMBOL,
    OHLC_DATA.VALUE:"from_date"::DATE                       OHLC_DATE,
    OHLC_DATA.VALUE:"open"::NUMBER(14, 4)                   OPEN_PRICE,
    OHLC_DATA.VALUE:"high"::NUMBER(14, 4)                   HIGH_PRICE,
    OHLC_DATA.VALUE:"low"::NUMBER(14,4)                     LOW_PRICE,
    OHLC_DATA.VALUE:"close"::NUMBER(14, 4)                  CLOSE_PRICE,
FROM 
    COMPANY_STOCK_PRICES_DAILY_OHLC_RAW_TBL CSTR,
    LATERAL FLATTEN (input => CSTR.RESULTS) OHLC_DATA
ORDER BY TICKER_SYMBOL, OHLC_DATE;

## Create a View on the Previous SQL to Prepare for Presentation To the Semantic Model Generator
```
CREATE OR REPLACE VIEW COMPANY_STOCK_PRICES_OHLC_VW
AS
SELECT
    TICKER::VARCHAR                                         TICKER_SYMBOL,
    OHLC_DATA.VALUE:"from_date"::DATE                       OHLC_DATE,
    TO_NUMBER(OHLC_DATA.VALUE:"open",14, 4)                 OPEN_PRICE,
    TO_NUMBER(OHLC_DATA.VALUE:"high",14, 4)                 HIGH_PRICE,
    TO_NUMBER(OHLC_DATA.VALUE:"low",14, 4)                  LOW_PRICE,
    TO_NUMBER(OHLC_DATA.VALUE:"close",14, 4)                CLOSE_PRICE,
FROM 
    COMPANY_STOCK_PRICES_DAILY_OHLC_RAW_TBL CSTR,
    LATERAL FLATTEN (input => CSTR.RESULTS) OHLC_DATA
ORDER BY TICKER_SYMBOL, OHLC_DATE;
```

In [ ]:
CREATE OR REPLACE VIEW COMPANY_STOCK_PRICES_OHLC_VW
AS
SELECT
    TICKER::VARCHAR                                         TICKER_SYMBOL,
    OHLC_DATA.VALUE:"from_date"::DATE                       OHLC_DATE,
    TO_NUMBER(OHLC_DATA.VALUE:"open",14, 4)                 OPEN_PRICE,
    TO_NUMBER(OHLC_DATA.VALUE:"high",14, 4)                 HIGH_PRICE,
    TO_NUMBER(OHLC_DATA.VALUE:"low",14, 4)                  LOW_PRICE,
    TO_NUMBER(OHLC_DATA.VALUE:"close",14, 4)                CLOSE_PRICE,
FROM 
    COMPANY_STOCK_PRICES_DAILY_OHLC_RAW_TBL CSTR,
    LATERAL FLATTEN (input => CSTR.RESULTS) OHLC_DATA
ORDER BY TICKER_SYMBOL, OHLC_DATE;

## Test the View

In [ ]:
SELECT * FROM COMPANY_STOCK_PRICES_OHLC_VW LIMIT 20;

## Before we create a semantic model for our view, we need a stage to store the model file. 
### Create a Stage - YAML_STAGE - for the Semantic Model File
#### If you already have a stage use it.  
```
CREATE STAGE YAML_STAGE 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );
```

## Create a Semantic Model for the OHLC Data. 
### This semantic model will be used by Cortex Analyst for generating queries. 
### We will use the Snowflake AI & ML Studio in Snowsight to generate the semantic model.
### Copy [this Semantic Model YAML file](https://github.com/rrprasan/Finance/blob/main/Snowflake/Notebooks/Company_Financials/unified_equity_research/ohlc_data_semantic_model.yaml) from Github into YAML_STAGE.
### Or, if you wish to create the semantic model file from scratch, please follow the [instructions in this page](https://github.com/rrprasan/Finance/blob/main/Snowflake/Notebooks/Company_Financials/unified_equity_research/Readme.md) to create the Semantic Model file.  

In [ ]:
CREATE STAGE YAML_STAGE 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

# What we have accomplished thus far? 
### 1. Ingested OHLC JSON data into Snowflake.
### 2. Transform the JSON Data using FLATTEN and CAST the data to the correct data type.
### 3. Created a VIEW on the flattened data. 
### 4. Created a Cortex Analyst Semantic Model on the View. 
### 5. Tested the semantic model with a few questions.
### 6. Recieved tabular data from Cortex Analyst.
# What are the remaining steps?
### 1. Create a stage for the company-related PDF files.
### 2. Store the company annual report PDF into a stage. 
### 3. Parse the PDF files in the stage and extract the text. 
### 4. Store the text in a VARCHAR column in a table. 
### 5. Split the text into chunks. 
### 6. Create Snowflake Cortex Search service on the chunks.
### 7. Test the Cortex Search Service to see it returns the correct results.  
### 8. Set-up Snowflake Cortex Agent Streamlit App.
### 9. Ask questions & get answers. 
### 10. Snowflake Cortex Agent will interpret your question. 
### 11. Route it intelligently to either Cortex Analyst or Cortex Search. 
### 12. If a question is routed to Cortex Analyst, tabular data is received as an answer. 
### 13. If a question is routed to Cortex Search,  textual answer is received as an answer.  

## Create a Stage to Store All the Company Related Filings, News, Analyst Reports
```
CREATE STAGE COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_STG 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );
```

In [ ]:
CREATE STAGE COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_STG 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

## Download the [Annual_Report_and_Earnings_Call_Transcript_PDF_Files.zip](https://github.com/rrprasan/Finance/tree/main/Snowflake/Notebooks/Company_Financials/unified_equity_research) file from Github.
## Unzip the files
## Upload the following file to the stage
- CPB_Earnings_Call_Transcript_Q2_2025_5_March_2025_8_00_AM_ET.pdf
- CPB_2024_Annual_Report.pdf

## Create a Table to Store the text from the PDF Obtained by Using the PARSE_DOCUMENT function.  
```
CREATE OR REPLACE TRANSIENT TABLE COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_TBL
(
    TICKER_SYMBOL VARCHAR,
    UPDATE_WEEK DATE,
    UPDATE_QUARTER VARCHAR, 
    UPDATE_TITLE VARCHAR,
    UPDATE_MESSAGE VARCHAR
);
```

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_TBL
(
    TICKER_SYMBOL VARCHAR,
    UPDATE_WEEK DATE,
    UPDATE_QUARTER VARCHAR, 
    UPDATE_TITLE VARCHAR,
    UPDATE_MESSAGE VARCHAR
);

## 1. Parse the CPB Earnings Call Transcript. 
## 2. Store text in COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_TBL Table.  

In [ ]:
INSERT INTO COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_TBL (TICKER_SYMBOL, UPDATE_WEEK, UPDATE_QUARTER, UPDATE_TITLE, UPDATE_MESSAGE)
SELECT 
    'CPB',
    TO_DATE('03-05-2025', 'MM-DD-YYYY') UPDATE_WEEK,
    'Q2 FY 2025',
    'Q2 FY 2025 Earnings Call Transcript',
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@DEMODB.EQUITY_RESEARCH.COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_STG',
                                      'CPB_Earnings_Call_Transcript_Q2_2025_5_March_2025_8_00_AM_ET.pdf',
                                      {'mode': 'LAYOUT'}):content;


## 1. Parse the CPB Annual Report PDF document. 
## 2. Store text in COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_TBL Table.

In [ ]:
INSERT INTO COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_TBL (TICKER_SYMBOL, UPDATE_WEEK, UPDATE_QUARTER, UPDATE_TITLE, UPDATE_MESSAGE)
SELECT 
    'CPB',
    TO_DATE('03-05-2025', 'MM-DD-YYYY') UPDATE_WEEK,
    '2024 10-K Annual Report',
    '2024 Annual Report',
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@DEMODB.EQUITY_RESEARCH.COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_STG',
                                      'CPB_2024_Annual_Report.pdf',
                                      {'mode': 'LAYOUT'}):content;

In [ ]:
DESC TABLE COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_TBL;

In [ ]:
SELECT * FROM COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_TBL;

## Create a Sequence to Generate Unique IDs for Each Text Chunk.
## This unique ID will be used to retrieve and create the citations.   

In [ ]:
CREATE OR REPLACE SEQUENCE COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_CHUNKS_SEQ;

### Create Text Chunks of the Content in the Annual Report and the Earnings Call Transcript. 
### Use SPLIT_TEXT_RECURSIVE_CHARACTER to create the chunks. 
### Store the text chunks in a new 'chunks' table - COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_CHUNKS_TBL.  

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_CHUNKS_TBL 
AS
SELECT
    COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_CHUNKS_SEQ.NEXTVAL UPDATE_CHUNK_ID,
    TICKER_SYMBOL,
    UPDATE_WEEK,
    UPDATE_QUARTER,
    UPDATE_TITLE,
    UPDATE_MESSAGE,
    CONCAT(UPDATE_QUARTER, '-', TO_VARCHAR(UPDATE_CHUNK_TXT.VALUE)) AS UPDATE_CHUNK
FROM
   COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_TBL,
   LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
      UPDATE_MESSAGE,
      'none',
      500,
      100
   )) UPDATE_CHUNK_TXT;

## Test the Chunks Table. 

In [ ]:
SELECT * FROM COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_CHUNKS_TBL LIMIT 20;

## Create Cortex Search Service

In [ ]:
CREATE CORTEX SEARCH SERVICE COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_SEARCH_SVC
    ON UPDATE_CHUNK
    ATTRIBUTES TICKER_SYMBOL, UPDATE_WEEK, UPDATE_QUARTER, UPDATE_TITLE
    WAREHOUSE = COMPUTE_WH
    TARGET_LAG = '1 hour'
    AS (
        SELECT *
        FROM COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_CHUNKS_TBL
    );

## Test the Cortex Search Service Using SEARCH_PREVIEW function in Snowflake. 

In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'COMPANY_ALL_FILINGS_NEWS_ANALYSIS_UPDATES_SEARCH_SVC',
      '{
        "query": "What are the risks to Campbell soup\'s business?",
        "columns":[
            "UPDATE_CHUNK"
        ],
        "limit":10
      }'
  )
)['results'] as results;

## Create a Streamlit App in Snowflake Snowsight. 
### Copy and paste [this code - Unified_Equity_Research_Streamlit.py](https://github.com/rrprasan/Finance/blob/main/Snowflake/Notebooks/Company_Financials/unified_equity_research/Unified_Equity_Research_Streamlit.py) - from Github into Streamlit in Snowflake.  
### Once you have made changes to the Streamlit app, you are ready to test Cortex Agent. 
### There are comments in the Streamlit app that tell you the changes you will have to make. 
This Streamlit code modifes the code found in the [Snowflake Cortex Agent quick-start guide](https://quickstarts.snowflake.com/guide/getting_started_with_cortex_agents/index.html?index=../..index#0).